In [7]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [8]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import when

spark.sql("USE 2023_04_01")
    # Assuming that 'ticker' is a valid Python variable

# query = f"""SELECT ticker_region FROM sym_ticker_region WHERE ticker_region LIKE "%-US" """
# df = spark.sql(query)
# df = df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
# ticker_list = df.collect()
# print(len(ticker_list))


DataFrame[]

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import when
import pyspark.pandas as ps
import numpy as np
from scipy.stats import zscore


#fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))

def query(ticker):
    query = f"""SELECT d.ticker_region, a.date
                FROM FF_ADVANCED_AF a 
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id 
                WHERE d.ticker_region = "{ticker}-US"
                ORDER BY a.date
                """

    fund_df = spark.sql(query)
    fund_df = fund_df.withColumn("ticker_region", regexp_replace("ticker_region", "-US$", ""))
    
    return fund_df

def get_top_bottom_ten(df):
    df = df.sort_values(by='Value')
    df=df.dropna()
    top10 = df.head(10)
    down10 = df.tail(10)
    print(top10,down10)
    return top10['Metric'].tolist(), down10['Metric'].tolist()

def pct_change_df(df, big_string, big_string2):
    df=spark.createDataFrame(df)
    df.createOrReplaceTempView("temp_table")
    
    query1 = f"""
                SELECT {big_string}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON t.Ticker = SUBSTRING(s.ticker_region, 1, LENGTH(s.ticker_region)-3) AND SUBSTRING(s.ticker_region, -2, 3) = 'US'
                LEFT JOIN FF_ADVANCED_AF a ON s.fsym_id = a.fsym_id AND YEAR(a.date) = YEAR(t.Implosion_Date)
                ORDER BY t.Ticker
            """
    query2 = f"""
                SELECT {big_string2}
                FROM temp_table t  
                LEFT JOIN sym_ticker_region s ON t.Ticker = SUBSTRING(s.ticker_region, 1, LENGTH(s.ticker_region)-3) AND SUBSTRING(s.ticker_region, -2, 3) = 'US'
                LEFT JOIN FF_ADVANCED_AF b ON s.fsym_id = b.fsym_id AND YEAR(b.date) = YEAR(t.Implosion_Prev4Years)
                ORDER BY t.Ticker
            """
    df1 = spark.sql(query1)
    df2 = spark.sql(query2)
    
    df1 = df1.toPandas()
    df2 = df2.toPandas()
    
    non_string_columns = df1.select_dtypes(exclude=['object']).columns
    df1 = df1[non_string_columns]
    df2 = df2[non_string_columns]
    
    null_threshold = 200
    columns_to_drop = df1.columns[df1.isnull().sum() > null_threshold]
    df1 = df1.drop(columns=columns_to_drop)
    df2 = df2.drop(columns=columns_to_drop)
    # print("NULLS:")
    # print(df1.isnull().sum())
    # print(df2.isnull().sum())
    
    percentage_change_df = ((df1 - df2) / df2) * 100
    #print(percentage_change_df)
    #print("LENGTH: ",len(percentage_change_df))
    
    
    metric_dict = {}
    for column in percentage_change_df.columns:
        percentage_change_df[column] = percentage_change_df[column].replace([np.inf, -np.inf], np.nan)
        new_col = percentage_change_df[column].dropna()
        mean_val = new_col.mean()
        stddev_val = new_col.std()
        z_score_threshold = 3.0
        new_col = new_col[(new_col >= mean_val - z_score_threshold * stddev_val) &
        (new_col <= mean_val + z_score_threshold * stddev_val)]
        #if new_col.std() < 1000:
        metric_dict[column] = new_col.mean()
    #print(metric_dict)
    metric_df = pd.DataFrame(list(metric_dict.items()), columns=['Metric', 'Value'])
    #metric_df.to_csv('ChangesBeforeImplosionA4yrs.csv', index=False)
    return metric_df
    
    # df['pct_change'] = (df[metric_curr] - df[metric_prev])/df[metric_prev]
    # df['pct_change'] = df['pct_change'].replace([np.inf, -np.inf], np.nan) 
    # df=df.dropna(axis=0)
    # mean_val = df['pct_change'].mean()
    # stddev_val = df['pct_change'].std()
    # z_score_threshold = 3.0
    # df = df[
    # (df['pct_change'] >= mean_val - z_score_threshold * stddev_val) &
    # (df['pct_change'] <= mean_val + z_score_threshold * stddev_val)]
    # new_mean = df['pct_change'].mean()
    # return new_mean




def get_metric_changes(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df_metrics = spark.sql("SELECT * FROM FF_ADVANCED_AF LIMIT 10")
    df_metrics = df_metrics.columns
    result_string = ', '.join('a.' + item for item in df_metrics)
    result_string2 = ', '.join('b.' + item for item in df_metrics)
    top10s = []
    bottom10s = []
    for y in range(1,5):
        df['Implosion_Prev4Years'] = df['Implosion_Date'] - pd.DateOffset(years=y)
    #print(result_string)
        new_df = pct_change_df(df, result_string, result_string2)
        top10, bottom10 = get_top_bottom_ten(new_df)
        top10s.append(top10)
    intersection1 = sorted(list(set(top10s[0]).intersection(top10s[1])))
    print(intersection1)
    intersection1 = sorted(list(set(top10s[0]).intersection(top10s[2])))
    print(intersection1)
    intersection1 = sorted(list(set(top10s[0]).intersection(top10s[3])))
    print(intersection1)
    intersection1 = sorted(list(set(top10s[1]).intersection(top10s[2])))
    print(intersection1)
    intersection1 = sorted(list(set(top10s[1]).intersection(top10s[3])))
    print(intersection1)
    intersection1 = sorted(list(set(top10s[2]).intersection(top10s[3])))
    print(intersection1)

#get_metric_changes('imploded_tickers_dates.csv')




/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [19]:
def consistent_changes(ticker, big_string):
    start_date = pd.to_datetime("2009-01-01")
    query1 = f"""
                SELECT a.date, b.FF_PRICE_CLOSE_FP
                FROM FF_ADVANCED_AF a
                LEFT JOIN sym_ticker_region d ON d.fsym_id = a.fsym_id 
                LEFT JOIN FF_BASIC_QF b ON b.fsym_id = a.fsym_id
                WHERE d.ticker_region = "{ticker}-US" 
                AND a.date >= "{start_date}"
                ORDER BY a.date
            """
    q_df = spark.sql(query1)
    q_df = ps.DataFrame(q_df)
    print(q_df)
    non_string_columns = q_df.select_dtypes(exclude=['object']).columns
    q_df = q_df[non_string_columns]
    # null_threshold = 10
    # columns_to_drop = q_df.columns[q_df.isnull().sum() > null_threshold]
    # q_df = q_df.drop(columns=columns_to_drop)
    correlations = q_df.corr()['FF_PRICE_CLOSE_FP']
    return correlations

def corr_analysis(filename):
    df = pd.read_csv(filename, index_col=False)
    df['Implosion_Date'] = pd.to_datetime(df['Implosion_Date']).dt.date
    df_metrics = spark.sql("SELECT * FROM ff_advanced_qf LIMIT 10")
    df_metrics = df_metrics.columns[:50]
    result_string = ', '.join('a.' + item for item in df_metrics)
    result_string2 = ', '.join('b.' + item for item in df_metrics)
    print(len(df_metrics))
    corr_matrices = []
    for t in df['Ticker'].unique().tolist():
        print(t)
        mat = consistent_changes(t, result_string)
        if not(mat.empty):
            corr_matrices.append(consistent_changes(t, result_string))
    avg_dict = {}
    df_metrics = corr_matrices[0].index.tolist()
    corr_count = 0
    for metric in df_metrics:
        met_values = [corr_mat[metric] for corr_mat in corr_matrices if not pd.isna(corr_mat[metric])]
        met_count = len(met_values)

        if met_count != 0:
            avg_dict[metric] = np.mean(met_values)
            corr_count += 1

        if corr_count % 25 == 0:
            metric_df = pd.DataFrame(list(avg_dict.items()), columns=['Metric', 'Value'])
            metric_df.to_csv("Avg_CorrelationsADVANCEDQF.csv")
        
        
    print(avg_dict)
    metric_df = pd.DataFrame(list(avg_dict.items()), columns=['Metric', 'Value'])
    metric_df.to_csv("Avg_CorrelationsADVANCEDQF.csv")
        
    
    
    
corr_analysis('imploded_tickers_dates.csv')

50
EMCMF
          date  FF_PRICE_CLOSE_FP
0   2018-12-31           0.726803
1   2018-12-31           0.891117
2   2018-12-31           0.560445
3   2018-12-31           0.499095
4   2018-12-31           0.490836
5   2018-12-31           0.557477
6   2018-12-31           0.503525
7   2018-12-31           0.499021
8   2018-12-31           0.510172
9   2018-12-31           0.489476
10  2018-12-31           0.514106
11  2018-12-31           0.515756
12  2018-12-31           0.464353
13  2018-12-31           0.468379
14  2018-12-31           0.507639
15  2018-12-31           0.517097
16  2018-12-31           0.145555
17  2018-12-31           0.158921
18  2019-12-31           0.726803
19  2019-12-31           0.891117
20  2019-12-31           0.560445
21  2019-12-31           0.499095
22  2019-12-31           0.490836
23  2019-12-31           0.557477
24  2019-12-31           0.503525
25  2019-12-31           0.499021
26  2019-12-31           0.510172
27  2019-12-31           0.489476
28  2

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [2]:
def get_top_bottom_five(df):
    df = df.sort_values(by='Value')
    df=df.dropna()
    top5 = df.head(10)
    down5 = df.tail(10)
    print(top5)
    print(down5)
    print(len(df))


df = pd.read_csv('Avg_Correlations.csv', index_col=None)
get_top_bottom_five(df)
#YOU'VE DONE WORST CHANGES NOW FIND OUT WHICH ONES DECREASE CONSISTENTLY
#ALSO FIGURE OUT MEANS BEFORE PERIOD AND AFTER PERIOD USING QUARTERLY AND COMPARE DIFF

    Unnamed: 0                Metric     Value
7            7        ff_ptx_xord_cr -0.351692
19          19  ff_assets_oth_intang -0.317973
3            3       ff_oper_exp_oth -0.304189
11          11         ff_eq_aff_inc -0.264472
23          23            ff_debt_st -0.248122
30          30               ff_debt -0.242477
22          22         ff_liabs_curr -0.096941
24          24           ff_pay_acct  0.020510
16          16      ff_bk_invest_tot  0.046414
17          17              ff_inven  0.062547
    Unnamed: 0             Metric     Value
1            1           ff_sales  0.285308
13          13         ff_div_pfd  0.295916
28          28         ff_pfd_stk  0.318619
2            2       ff_gross_inc  0.327146
15          15       ff_cash_only  0.327874
29          29          ff_com_eq  0.337377
31          31       ff_shldrs_eq  0.337402
18          18     ff_assets_curr  0.366452
5            5        ff_oper_inc  0.400717
0            0  FF_PRICE_CLOSE_FP  1.000000

In [ ]:
metric_df

In [ ]:
metric_dict